<a href="https://colab.research.google.com/github/FaizanHameed1/Text-Classification-with-Multi-Layer-Perceptiron-MLP-based-Siamese-Neaural-Network-Model/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
# Visualize Tree
from sklearn.tree import export_graphviz

import os
import numpy as np
import tqdm
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_path="/content/drive/MyDrive/Kaggle/siamese/train_randomForest.csv"
test_path="/content/drive/MyDrive/Kaggle/siamese/test_randomForest.csv"

In [ ]:
df_train=pd.read_csv(train_path)
df_test=pd.read_csv(test_path)
df_train.head()

,label,embedding
0,6,[ 1.34379387e-01 1.02851443e-01 2.75034979e-...
1,6,[ 1.34379387e-01 1.02851443e-01 2.75034979e-...
2,1,[ 1.34379387e-01 1.02851450e-01 2.75034960e-...
3,1,[ 1.34379387e-01 1.02851443e-01 2.75034960e-...
4,1,[ 1.34379372e-01 1.02851450e-01 2.75035016e-...


In [ ]:
#for train set
raw_list_train = []
for item in df_train['embedding']:
  z = item.replace('\n', "").replace('  ', ' ').strip('][').split(' ')
  raw_list_train.append(z)
print(raw_list_train[0])
print(raw_list_train[2])
#for test set
raw_list_test = []
for item in df_test['embedding']:
  z = item.replace('\n', "").replace('  ', ' ').strip('][').split(' ')
  raw_list_test.append(z)
print(raw_list_test[0])
print(raw_list_test[100])

['', '1.34379387e-01', '1.02851443e-01', '2.75034979e-02', '1.09724896e-02', '-5.73562980e-02', '-4.17979173e-02', '-4.86538820e-02', '9.76219401e-03', '-5.20076975e-02', '-1.25665748e-02', '-1.60140973e-02', '-1.37981372e-02', '-1.23516828e-01', '-2.71983147e-02', '-2.00934950e-02', '-3.12091801e-02', '5.50912246e-02', '-4.68309857e-02', '-4.49433140e-02', '1.66746885e-01', '-9.68224835e-03', '-1.25090227e-01', '-4.42459807e-03', '1.09775044e-01', '9.10385326e-02', '8.44281167e-02', '-4.93285321e-02', '-2.47102812e-01', '-1.60937056e-01', '-6.14018664e-02', '-6.78255176e-03', '-5.99245094e-02', '3.37911700e-03', '-8.21462572e-02', '2.14735046e-02', '1.32628111e-02', '-1.17389839e-02', '-1.09540470e-01', '5.23472996e-03', '-1.95723668e-01', '1.39980568e-02', '-2.49633449e-03', '1.44459689e-02', '1.11682462e-02', '-7.75540993e-02', '-1.01943277e-01', '-4.15790863e-02', '4.46198843e-02', '-8.33148137e-03', '-2.73501307e-01', '1.34277046e-02', '2.95415539e-02', '3.71108204e-02', '-6.08297

In [ ]:
print(len(raw_list_train))
print(len(raw_list_test))

11415
1669


In [ ]:
print(raw_list_train[0])
print(len(raw_list_train[0]))

['', '1.34379387e-01', '1.02851443e-01', '2.75034979e-02', '1.09724896e-02', '-5.73562980e-02', '-4.17979173e-02', '-4.86538820e-02', '9.76219401e-03', '-5.20076975e-02', '-1.25665748e-02', '-1.60140973e-02', '-1.37981372e-02', '-1.23516828e-01', '-2.71983147e-02', '-2.00934950e-02', '-3.12091801e-02', '5.50912246e-02', '-4.68309857e-02', '-4.49433140e-02', '1.66746885e-01', '-9.68224835e-03', '-1.25090227e-01', '-4.42459807e-03', '1.09775044e-01', '9.10385326e-02', '8.44281167e-02', '-4.93285321e-02', '-2.47102812e-01', '-1.60937056e-01', '-6.14018664e-02', '-6.78255176e-03', '-5.99245094e-02', '3.37911700e-03', '-8.21462572e-02', '2.14735046e-02', '1.32628111e-02', '-1.17389839e-02', '-1.09540470e-01', '5.23472996e-03', '-1.95723668e-01', '1.39980568e-02', '-2.49633449e-03', '1.44459689e-02', '1.11682462e-02', '-7.75540993e-02', '-1.01943277e-01', '-4.15790863e-02', '4.46198843e-02', '-8.33148137e-03', '-2.73501307e-01', '1.34277046e-02', '2.95415539e-02', '3.71108204e-02', '-6.08297

In [ ]:
#cleaning data for train set
final_list_train=[]
for lst in raw_list_train:
  sub_list=[]
  for item in lst:
    if item!='':
      sub_list.append(float(item))
    else:
      continue
  final_list_train.append(sub_list)
print(final_list_train[0])
print(final_list_train[2])

#cleaning data for test set
final_list_test=[]
for lst in raw_list_test:
  sub_list=[]
  for item in lst:
    if item!='':
      sub_list.append(float(item))
    else:
      continue
  final_list_test.append(sub_list)
print(final_list_test[0])
print(final_list_test[100])

 

[0.134379387, 0.102851443, 0.0275034979, 0.0109724896, -0.057356298, -0.0417979173, -0.048653882, 0.00976219401, -0.0520076975, -0.0125665748, -0.0160140973, -0.0137981372, -0.123516828, -0.0271983147, -0.020093495, -0.0312091801, 0.0550912246, -0.0468309857, -0.044943314, 0.166746885, -0.00968224835, -0.125090227, -0.00442459807, 0.109775044, 0.0910385326, 0.0844281167, -0.0493285321, -0.247102812, -0.160937056, -0.0614018664, -0.00678255176, -0.0599245094, 0.003379117, -0.0821462572, 0.0214735046, 0.0132628111, -0.0117389839, -0.10954047, 0.00523472996, -0.195723668, 0.0139980568, -0.00249633449, 0.0144459689, 0.0111682462, -0.0775540993, -0.101943277, -0.0415790863, 0.0446198843, -0.00833148137, -0.273501307, 0.0134277046, 0.0295415539, 0.0371108204, -0.0608297177, 0.175936967, -0.00942696538, -0.182766423, -0.0296990667, 0.0980092809, -0.108096823, -0.031061437, 0.0784531683, -0.0663472041, -0.049624417, 0.0547899716, -0.0101606399, -0.00547771389, -0.0316788852, -0.0948420092, -0.

In [ ]:
print(len(final_list_train))
print(len(final_list_train[0]))
print(len(final_list_test))
print(len(final_list_test[0]))

11415
768
1669
768


In [ ]:
train_features=pd.DataFrame(final_list_train)
test_features=pd.DataFrame(final_list_test)
print(train_features.head())
print(test_features.head())

        0         1         2    ...       765       766       767
0  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
1  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
2  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
3  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
4  0.134379  0.102851  0.027504  ...  0.024879 -0.026197  0.052857

[5 rows x 768 columns]
        0         1         2    ...       765       766       767
0  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
1  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
2  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
3  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857
4  0.134379  0.102851  0.027503  ...  0.024879 -0.026197  0.052857

[5 rows x 768 columns]


In [ ]:
train_labels=df_train['label']
test_labels=df_test['label']
print(train_labels.head())
print(test_labels.head())

0    6
1    6
2    1
3    1
4    1
Name: label, dtype: int64
0    1
1    1
2    1
3    1
4    1
Name: label, dtype: int64


In [ ]:
print(train_features.isnull().sum().any())#checking null values
print(test_features.isnull().sum().any())#checking null values)

False
False


In [ ]:
train_features.dtypes

0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
763    float64
764    float64
765    float64
766    float64
767    float64
Length: 768, dtype: object

In [ ]:
full_train_df=pd.concat([train_features, train_labels], axis=1, join='inner')
full_train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,label
0,0.134379,0.102851,0.027503,0.010972,-0.057356,-0.041798,-0.048654,0.009762,-0.052008,-0.012567,-0.016014,-0.013798,-0.123517,-0.027198,-0.020093,-0.031209,0.055091,-0.046831,-0.044943,0.166747,-0.009682,-0.12509,-0.004425,0.109775,0.091039,0.084428,-0.049329,-0.247103,-0.160937,-0.061402,-0.006783,-0.059925,0.003379,-0.082146,0.021474,0.013263,-0.011739,-0.10954,0.005235,-0.195724,...,0.025771,0.167979,0.075221,0.000593,0.15223,0.173133,-0.100758,-0.043478,-0.052384,-0.166208,-0.06363,-0.003947,-0.000834,0.070546,0.022969,0.009174,-0.059754,-0.104772,-0.002437,0.032185,0.188568,0.092479,-0.01411,-0.025252,0.033435,-0.043951,0.022084,-0.103522,0.030269,-0.054122,-0.048553,-0.119989,0.038288,0.004404,-0.010769,0.003679,0.024879,-0.026197,0.052857,6.0
1,0.134379,0.102851,0.027503,0.010972,-0.057356,-0.041798,-0.048654,0.009762,-0.052008,-0.012567,-0.016014,-0.013798,-0.123517,-0.027198,-0.020093,-0.031209,0.055091,-0.046831,-0.044943,0.166747,-0.009682,-0.12509,-0.004425,0.109775,0.091039,0.084428,-0.049329,-0.247103,-0.160937,-0.061402,-0.006783,-0.059925,0.003379,-0.082146,0.021474,0.013263,-0.011739,-0.10954,0.005235,-0.195724,...,0.025771,0.167979,0.075221,0.000593,0.15223,0.173133,-0.100758,-0.043478,-0.052384,-0.166208,-0.06363,-0.003947,-0.000834,0.070546,0.022969,0.009174,-0.059754,-0.104772,-0.002437,0.032185,0.188568,0.092479,-0.01411,-0.025252,0.033435,-0.043951,0.022084,-0.103522,0.030269,-0.054122,-0.048553,-0.119989,0.038288,0.004404,-0.010769,0.003679,0.024879,-0.026197,0.052857,6.0
2,0.134379,0.102851,0.027503,0.010972,-0.057356,-0.041798,-0.048654,0.009762,-0.052008,-0.012567,-0.016014,-0.013798,-0.123517,-0.027198,-0.020093,-0.031209,0.055091,-0.046831,-0.044943,0.166747,-0.009682,-0.12509,-0.004425,0.109775,0.091039,0.084428,-0.049329,-0.247103,-0.160937,-0.061402,-0.006783,-0.059925,0.003379,-0.082146,0.021474,0.013263,-0.011739,-0.10954,0.005235,-0.195724,...,0.025771,0.167979,0.075221,0.000593,0.15223,0.173133,-0.100758,-0.043478,-0.052384,-0.166208,-0.06363,-0.003947,-0.000834,0.070546,0.022969,0.009174,-0.059754,-0.104772,-0.002437,0.032185,0.188568,0.092479,-0.01411,-0.025252,0.033435,-0.043951,0.022084,-0.103522,0.030269,-0.054122,-0.048553,-0.119990,0.038288,0.004404,-0.010769,0.003679,0.024879,-0.026197,0.052857,1.0
3,0.134379,0.102851,0.027503,0.010972,-0.057356,-0.041798,-0.048654,0.009762,-0.052008,-0.012567,-0.016014,-0.013798,-0.123517,-0.027198,-0.020093,-0.031209,0.055091,-0.046831,-0.044943,0.166747,-0.009682,-0.12509,-0.004425,0.109775,0.091039,0.084428,-0.049329,-0.247103,-0.160937,-0.061402,-0.006783,-0.059925,0.003379,-0.082146,0.021474,0.013263,-0.011739,-0.10954,0.005235,-0.195724,...,0.025771,0.167979,0.075221,0.000593,0.15223,0.173133,-0.100758,-0.043478,-0.052384,-0.166208,-0.06363,-0.003947,-0.000834,0.070546,0.022969,0.009174,-0.059754,-0.104772,-0.002437,0.032185,0.188568,0.092479,-0.01411,-0.025252,0.033435,-0.043951,0.022084,-0.103522,0.030269,-0.054122,-0.048553,-0.119989,0.038288,0.004404,-0.010769,0.003679,0.024879,-0.026197,0.052857,1.0
4,0.134379,0.102851,0.027504,0.010972,-0.057356,-0.041798,-0.048654,0.009762,-0.052008,-0.012567,-0.016014,-0.013798,-0.123517,-0.027198,-0.020093,-0.031209,0.055091,-0.046831,-0.044943,0.166747,-0.009682,-0.12509,-0.004425,0.109775,0.091039,0.084428,-0.049329,-0.247103,-0.160937,-0.061402,-0.006783,-0.059925,0.003379,-0.082146,0.021474,0.013263,-0.011739,-0.10954,0.005235,-0.195724,...,0.025771,0.167979,0.075221,0.000593,0.15223,0.173133,-0.100758,-0.043478,-0.052384,-0.166208,-0.06363,-0.003947,-0.000834,0.070546,0.022969,0.009174,-0.059754,-0.104772,-0.002437,0.032185,0.188568,0.092479,-0.01411,-0.025252,0.033435,-0.043951,0.022084,-0.103522,0.030269,-0.054122,-0.0485

In [ ]:
full_train_df.dtypes

0        float64
1        float64
2        float64
3        float64
4        float64
          ...   
764      float64
765      float64
766      float64
767      float64
label    float64
Length: 769, dtype: object

In [ ]:
# full_train_df.describe()

In [ ]:
# full_train_df.corr()

In [ ]:
# #Using Pearson Correlation
# plt.figure(figsize=(12,10))
cor = full_train_df.corr()
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()

In [ ]:
cor

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,label
0,1.000000,-0.117889,0.012549,NaN,-0.035718,0.056856,-0.067986,0.005759,0.029144,0.054706,-0.110449,0.022135,-0.144100,0.002812,-0.021453,-0.112102,-0.046199,0.111475,-0.071945,-0.088622,-0.175771,0.088726,0.036982,-0.112218,0.003711,0.212247,0.058185,-0.033183,-0.033868,-0.052670,-0.086405,-0.066014,0.068462,0.014541,0.037235,0.212664,0.065816,-0.051873,0.034778,0.013731,...,0.182297,-0.068685,-0.000535,-0.009769,0.227961,-0.061646,-0.145057,-0.024148,-0.037546,-0.115020,0.085557,0.115326,0.034334,0.054411,0.101377,0.067245,-0.020236,-0.053385,0.058304,0.004273,0.070344,0.151692,0.079413,0.040278,-0.116739,-0.055161,0.111372,-0.136040,-0.093391,-0.157251,-0.097074,0.081356,0.177366,0.157274,0.071495,0.160478,0.184556,0.057111,0.027214,-0.002816
1,-0.117889,1.000000,0.228048,NaN,-0.132093,-0.130593,0.034955,0.148126,-0.151297,0.099036,0.048510,0.047422,0.105567,-0.115300,-0.036279,0.002039,0.181467,-0.307206,-0.115925,0.337301,0.116459,-0.311636,-0.169385,0.359199,0.136784,-0.158310,-0.197064,-0.141428,-0.020331,-0.125795,0.273576,-0.100773,-0.154009,-0.210712,0.036258,-0.096546,-0.096587,-0.135668,-0.031239,-0.226754,...,-0.147112,0.279575,0.198085,0.198263,-0.100638,0.373321,0.192930,-0.014359,-0.177521,0.016726,-0.300100,-0.268866,-0.063618,0.176447,-0.045890,-0.096686,-0.193417,-0.175312,0.156788,0.257775,0.162752,0.040906,0.101335,-0.074886,0.374211,-0.151783,-0.038058,0.145992,0.194963,0.152376,-0.049730,-0.242270,-0.154591,-0.029081,0.020925,0.079075,-0.188651,-0.157793,0.181428,0.015611
2,0.012549,0.228048,1.000000,NaN,-0.158481,-0.061765,-0.052539,0.143779,-0.094874,0.162345,0.027414,0.045307,0.050330,-0.084112,-0.000414,-0.098822,0.082247,-0.203535,-0.250148,0.206019,-0.068976,-0.240492,-0.046817,0.244017,0.178124,-0.056342,-0.121745,-0.120940,-0.021573,-0.138198,0.262407,-0.136297,-0.104071,-0.194488,0.075469,0.021306,-0.031338,-0.116762,-0.072344,-0.274731,...,-0.071720,0.198023,0.174535,0.096223,0.000362,0.236227,0.107170,0.011436,-0.194153,-0.077811,-0.194851,-0.193526,-0.008181,0.164127,-0.064423,-0.102951,-0.077023,-0.127819,0.149828,0.190333,0.207883,0.154373,0.015701,-0.081331,0.193296,-0.137651,0.122512,-0.007465,0.077134,0.011738,-0.116634,-0.118951,-0.033625,0.054123,-0.005119,0.038988,-0.126388,-0.069376,0.178508,-0.024010
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.035718,-0.132093,-0.158481,NaN,1.000000,0.002428,0.057060,-0.102209,0.146463,0.027431,0.062914,0.094008,-0.020369,0.098988,0.042032,0.082659,-0.056269,0.087385,0.158878,-0.140806,0.050592,0.123437,0.117595,-0.122018,-0.124262,-0.008023,0.068870,0.081507,0.018809,0.172048,-0.097435,0.031156,0.216590,0.097432,0.042960,-0.045261,0.060961,0.062467,0.171590,0.186353,...,-0.023665,-0.067698,-0.139403,-0.035800,-0.096623,-0.120098,0.237575,0.076767,0.136746,0.069196,0.092907,0.122986,0.112892,-0.092468,0.035718,0.133536,0.077791,0.106069,-0.075586,-0.096669,-0.104752,-0.080137,-0.007483,0.064752,-0.106144,0.078152,-0.058025,0.022835,-0.070871,-0.021479,0.168553,0.065650,-0.058332,0.015700,-0.012529,0.097987,0.009814,0.086410,-0.082093,0.024620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,0.160478,0.079075,0.038988,N

In [ ]:
#Correlation with output variable
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
relevant_features

In [ ]:
#normalizing
# scaler=StandardScaler()
scaler=MinMaxScaler()
knn_train_features=scaler.fit_transform(train_features)
knn_test_features=scaler.fit_transform(test_features)

In [ ]:
len(knn_train_features[0])

768

In [ ]:
print(len(knn_train_features))
print(len(train_labels))
print(len(knn_test_features))
print(len(test_labels))

11415
11415
1669
1669


In [ ]:
# train_features.iloc[:1,:]

In [ ]:
# knn_train_features[0]

In [ ]:
print(f"Unique Labels after Encoding:  {df_train.label.unique()}")

Unique Labels after Encoding:  [ 6  1 10  3  8  4  5  0  7 12  2  9 11]


In [ ]:
print(type(train_features))
print(type(train_labels))
print(type(test_features))
print(type(test_labels))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [ ]:
print('Shape of Train Features :', train_features.shape)
print('Shape of Test Features :', test_features.shape)
print('Shape of Train Labels :', train_labels.shape)
print('Shape of Test Labels :', test_labels.shape)

Shape of Train Features : (11415, 768)
Shape of Test Features : (1669, 768)
Shape of Train Labels : (11415,)
Shape of Test Labels : (1669,)


In [2]:
#applying KNN classifier
knn=KNeighborsClassifier()
knn.fit(knn_train_features,train_labels)
y_pr=knn.predict(knn_test_features)
cm = confusion_matrix(test_labels, y_pr)
cm

In [3]:
# classification report
print(classification_report(test_labels, y_pr))

# print(confusion_matrix(y_test, y_predict))

In [4]:
print(f"Accuracy Score: {(accuracy_score(test_labels, y_pr)) * 100} %" )